# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [1]:
from typing import Tuple, List

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState
from copy import copy
from random import choice
import ipywidgets as widgets

Wielkość planszy

In [2]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [3]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour, row_count: int, column_count: int):
        self.game = game
        self._row_count = row_count
        self._column_count = column_count

    def evaluate_position(self, board: ConnectFourState, player: Player)->float:
        prize = 0

        # Check verticals
        for i in range(self._column_count):
            for j in range(self._row_count - 3):
                segment = [board.fields[i][j+z] for z in range(4)]
                prize += self._evaluate_segment(segment, player)
                
        # Check horizontals
        for i in range(self._column_count - 3):
            for j in range(self._row_count):
                segment = [board.fields[i+z][j] for z in range(4)]
                prize += self._evaluate_segment(segment, player)

        # Check rising edge diagonals
        for i in range(self._column_count - 3):
            for j in range(self._row_count - 3):
                segment = [board.fields[i+z][j+z] for z in range(4)]
                prize += self._evaluate_segment(segment, player)

        # Check falling edge diagonals
        for i in range(self._column_count - 3):
            for j in range(3, self._row_count):
                segment = [board.fields[i+z][j-z] for z in range(4)]
                prize += self._evaluate_segment(segment, player)

        return prize
    
    def _evaluate_segment(self, segment: List, player: Player):
        opponent = self.game.second_player
        if player == self.game.second_player:
            opponent = self.game.first_player
        
        prize = 0
        
        center = 4
        two_in_line = 2
        three_in_line = 10
        winning_condition = 1000
        opp_two_in_line = -2
        opp_winning_condition = -100
        
        if segment.count(player) == 4:
            prize += winning_condition
        elif segment.count(player) == 3 and segment.count(None) == 1:
            prize += three_in_line
        elif segment.count(player) == 2 and segment.count(None) == 2:
            prize += two_in_line
        elif segment.count(opponent) == 3 and segment.count(None) == 1:
            prize += opp_winning_condition
        elif segment.count(opponent) == 2 and segment.count(None) == 2:
            prize += opp_two_in_line
        
        return prize
    
    def is_valid_move(self, col_index:int)->bool:
        if self.game.state.fields[col_index][-1] is None:
            return True
        else:
            return False

    def get_valid_moves(self):
        return [valid_column for valid_column in range(COLUMN_COUNT) if self.is_valid_move(valid_column)]

    def get_best_move(self, player: Player)->int:
        valid_moves = self.get_valid_moves()

        best_prize = 0
        best_move = choice(valid_moves)

        for move in valid_moves:
            board_copy = copy(self.game.state)
            board_copy = board_copy.make_move(ConnectFourMove(move))
            prize = self.evaluate_position(board_copy, player)

            print(f"Column: {move}\tPrize: {prize}")
            if prize > best_prize:
                best_prize = prize
                best_move = move

        return best_move


    def count_player(self, player: Player):
        """# Check verticals
        for i in range(self._column_count):
            for j in range(self._row_count - 3):
                segment = [self.game.state.fields[i][j+z] for z in range(4)]"""
        pass

    def is_terminal(self) -> bool:
        # Check verticals
        for i in range(self._column_count):
            for j in range(self._row_count - 3):
                segment = [self.game.state.fields[i][j+z] for z in range(4)]
                if segment.count(self.game.first_player) == 4 or segment.count(self.game.second_player) == 4:
                    return True

        # Check horizontals
        for i in range(self._column_count - 3):
            for j in range(self._row_count):
                segment = [self.game.state.fields[i+z][j] for z in range(4)]
                if segment.count(self.game.first_player) == 4 or segment.count(self.game.second_player) == 4:
                    return True

        # Check rising edge diagonals
        for i in range(self._column_count - 3):
            for j in range(self._row_count - 3):
                segment = [self.game.state.fields[i+z][j+z] for z in range(4)]
                if segment.count(self.game.first_player) == 4 or segment.count(self.game.second_player) == 4:
                    return True


        # Check falling edge diagonals
        for i in range(self._column_count - 3):
            for j in range(3, self._row_count):
                segment = [self.game.state.fields[i+z][j-z] for z in range(4)]
                if segment.count(self.game.first_player) == 4 or segment.count(self.game.second_player) == 4:
                    return True

        return False

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        valid_moves = self.get_valid_moves()
        is_terminal = self.is_terminal()
        if is_terminal or depth == 0:
            if is_terminal:
                if self.game.get_winner() == self.game.first_player:
                    return (None, 100000)
                elif self.game.get_winner() == self.game.second_player:
                    return (None, -10000)
                else:
                    return (None, 0)
            else:
                return (self.get_best_move(), self.evaluate_position(self.game.second_player))

        if is_maximizing_player:
            for valid_move in valid_moves:
                # TODO Zaimplementować wykorzystywanie poprawnego ruchu

                alpha = max(alpha, self.minimax(depth-1, alpha, beta, not is_maximizing_player), key=lambda x: x[1])
                if alpha >= beta:
                    return alpha
            return alpha

        else:
            for valid_move in valid_moves:
                # TODO Zaimplementować wykorzystywanie poprawnego ruchu

                beta = min(beta, self.minimax(depth-1, alpha, beta, not is_maximizing_player), key=lambda x: x[1])
                if alpha >= beta:
                    return beta
            return beta

Rozgrywka

In [5]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
algorithm = MinMaxSolver(game, ROW_COUNT, COLUMN_COUNT)

while not algorithm.is_terminal():
    choosen_column = input("Column: ")
    #print(f"Playing player: {p1.char}")
    p1_best = algorithm.get_best_move(p1)
    game.make_move(ConnectFourMove(int(p1_best)))

    print(game)

    print(f"Playing AI: {p2.char}")
    p2_best = algorithm.get_best_move(p2)
    game.make_move(ConnectFourMove(p2_best))

    print(game)

print(f"Won: {game.get_winner().char}")
print("Terminated")

Column: 0	Prize: 0
Column: 1	Prize: 0
Column: 2	Prize: 0
Column: 3	Prize: 0
Column: 4	Prize: 0
Column: 5	Prize: 0
Column: 6	Prize: 0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
Playing AI: b
Column: 0	Prize: 0
Column: 1	Prize: 0
Column: 2	Prize: 0
Column: 3	Prize: 0
Column: 4	Prize: 0
Column: 5	Prize: 0
Column: 6	Prize: 0
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
Column: 0	Prize: 0
Column: 1	Prize: 0
Column: 2	Prize: 2
Column: 3	Prize: 2
Column: 4	Prize: 0
Column: 5	Prize: 0
Column: 6	Prize: 0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
Playing AI: b
Column: 0	Prize: -2
Column: 1	Prize: -2
Column: 2	Prize: 0
Column: 3	Prize: 0
Column: 4	Prize: 0
Column: 5	Prize: 0
Column: